In [133]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb
import mlflow
import joblib
import nltk
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from nltk.corpus import stopwords
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


In [134]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
    

In [135]:
from custom_transformers import FeatureAdder, NMFTransformer

In [136]:
pipeline = joblib.load("./email_pipeline.pkl")
print("Pipeline loaded successfully!")


Pipeline loaded successfully!


In [137]:
df=pd.read_csv("../Data/final_train.csv")

In [138]:
df.head()

,mail,type,clean_text,dominant_topic,dollar_count,exclamation_count,diacritic_count
0,"On Sun, 11 Aug 2002 bitbitch@magnesium.net wro...",0,sun 11 aug 2002 bitbitchmagnesiumnet wrote mea...,3,2,0,0
1,hello all : please send an email to : ibuyit @...,0,hello please send email ibuyit enron com s...,6,1,4,0
2,"idioms content - length : 99 please send "" app...",0,idioms content length 99 please send appala...,4,1,1,0
3,re : work from home this is one time message ....,1,work home one time message received email ex...,7,1,0,0
4,test results - resend by december 14 th the su...,1,test results resend december 14 th super pain...,5,1,1,0


In [139]:
X_train=df.drop(['type','mail','clean_text'],axis=1)
y_train=df['type']

In [140]:
X_train.head()

,dominant_topic,dollar_count,exclamation_count,diacritic_count
0,3,2,0,0
1,6,1,4,0
2,4,1,1,0
3,7,1,0,0
4,5,1,1,0


In [141]:
y_train

0        0
1        0
2        0
3        1
4        1
        ..
14605    1
14606    0
14607    0
14608    0
14609    0
Name: type, Length: 14610, dtype: int64

In [142]:
val=pd.read_csv("../Data/val.csv")

In [143]:
val

,mail,type
0,re : enron offsite hi steve : listed below is ...,Safe Email
1,market transparency ruling and bingaman bill p...,Safe Email
2,query on physical appearance of words content ...,Safe Email
3,get the best price on your next car ! exclusiv...,Phishing Email
4,re : fine with me - - - - - original message -...,Safe Email
...,...,...
1860,activities 1 . receivables backed finance : ti...,Safe Email
1861,resume of zach streight do you want me to take...,Safe Email
1862,best online pills here dickey bordello lick pa...,Phishing Email
1863,"same day change - hpl nom for november 11 , 20...",Safe Email


In [144]:
val=val.fillna('')

In [145]:
val=pipeline.transform(val)
val = pd.DataFrame(val, columns=[
    'dominant_topic','dollar_count', 'exclamation_count', 'clean_text', 'diacritic_count', 'mail', 'type'
])

In [146]:
val.head()

,dominant_topic,dollar_count,exclamation_count,clean_text,diacritic_count,mail,type
0,1,4,10,enron offsite hi steve listed information ne...,0,re : enron offsite hi steve : listed below is ...,Safe Email
1,0,0,0,market transparency ruling bingaman bill per l...,0,market transparency ruling and bingaman bill p...,Safe Email
2,0,0,0,query physical appearance words content lengt...,0,query on physical appearance of words content ...,Safe Email
3,4,0,1,get best price next car ! exclusive offer 24 x...,0,get the best price on your next car ! exclusiv...,Phishing Email
4,0,0,0,fine original message presto kevi...,0,re : fine with me - - - - - original message -...,Safe Email


In [147]:
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore',drop='first'), ['dominant_topic']),
    ],
    remainder='passthrough'
)

In [148]:
X_train_encoded = preprocessor.fit_transform(X_train)

In [149]:
X_train_encoded.shape

(14610, 12)

In [150]:
X_val=val.drop(['type','mail','clean_text'],axis=1)
y_val=val['type']

In [151]:
y_val=y_val.replace({'Phishing Email': 1, 'Safe Email': 0})

/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_40471/1528331634.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_val=y_val.replace({'Phishing Email': 1, 'Safe Email': 0})


In [152]:
X_val

,dominant_topic,dollar_count,exclamation_count,diacritic_count
0,1,4,10,0
1,0,0,0,0
2,0,0,0,0
3,4,0,1,0
4,0,0,0,0
...,...,...,...,...
1860,0,0,0,0
1861,9,0,0,0
1862,0,0,4,0
1863,0,0,0,0


In [153]:
X_val_encoded=preprocessor.transform(X_val)

In [154]:
X_val_encoded.shape

(1865, 12)

# Models

Let us try using different models to get good results  based on the designed feartures, we shall use the val set which has been preproccessed in the same way as train set has been pre-processed 

In [160]:
params={"n_estimators":100,"random_state":42,"n_jobs":-1}

In [161]:
# Initialize the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

# # Train the model on the training set
rf_model.fit(X_train_encoded, y_train)

RandomForestClassifier(n_jobs=-1, random_state=42)

In [162]:
y_pred = rf_model.predict(X_val_encoded)

# Print accuracy
acc=accuracy_score(y_val,y_pred)
print("Validation Accuracy:", acc)

# Print detailed classification report
report1=classification_report(y_val,y_pred)
report=classification_report(y_val,y_pred,output_dict=True)
print(report1)

Validation Accuracy: 0.3796246648793566
              precision    recall  f1-score   support

           0       0.48      0.22      0.31      1132
           1       0.34      0.62      0.44       733

    accuracy                           0.38      1865
   macro avg       0.41      0.42      0.37      1865
weighted avg       0.42      0.38      0.36      1865



In [163]:
mlflow.set_experiment("Phishing_Email_Detection")
mlflow.set_tracking_uri("http://127.0.0.1:5000")
with mlflow.start_run(run_name="Random_Forest"):
    mlflow.log_params(params)
    mlflow.log_metrics({
        'accuracy ':report['accuracy'],
        'recall_class_1 ': report['1']['recall'],
        'f1_score_macro':report['macro avg']['f1-score'],
        'f1_score_macro':report['weighted avg']['f1-score']

    })
    mlflow.sklearn.log_model(rf_model,"random_forest")

2025/02/04 20:51:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random_Forest at: http://127.0.0.1:5000/#/experiments/618310494445782914/runs/0139f289860940f89fa5f9f9901e52b2
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/618310494445782914


# Naive bayes 

In [118]:
df['clean_text']=df['clean_text'].fillna('')

In [119]:
# Initialize the Naive Bayes model
nb_model = MultinomialNB()

# Create a pipeline to combine the vectorization and the model
pipeline = make_pipeline(TfidfVectorizer(), nb_model)

# Fit the model on the training set
pipeline.fit(df['clean_text'], y_train)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('multinomialnb', MultinomialNB())])

In [120]:
val['clean_text']=val['clean_text'].fillna('')

In [122]:
# Predict on the validation set
y_pred_nb = pipeline.predict(val['clean_text'])


# Print detailed classification report
report1=classification_report(y_val,y_pred_nb)
report=classification_report(y_val,y_pred_nb,output_dict=True)
print(report1)

              precision    recall  f1-score   support

           0       0.92      0.98      0.95      1132
           1       0.97      0.87      0.92       733

    accuracy                           0.94      1865
   macro avg       0.95      0.93      0.93      1865
weighted avg       0.94      0.94      0.94      1865



In [123]:
with mlflow.start_run(run_name="Naive_Bayes"):
    mlflow.log_params({'vectorizer':"Tf-IDF vectorizer"})
    mlflow.log_metrics({
        'accuracy ':report['accuracy'],
        'recall_class_1 ': report['1']['recall'],
        'f1_score_macro':report['macro avg']['f1-score'],
        'f1_score_macro':report['weighted avg']['f1-score']

    })
    mlflow.sklearn.log_model(nb_model,"naive_bayes")

2025/02/04 20:43:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Naive_Bayes at: http://127.0.0.1:5000/#/experiments/618310494445782914/runs/d384b4020cb14faba114c35c1b5b4cbd
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/618310494445782914


## XGBOOST

In [171]:
dtrain = xgb.DMatrix(X_train_encoded, label=y_train)
dval = xgb.DMatrix(X_val_encoded, label=y_val)

# Define hyperparameters
params = {
    'objective': 'binary:logistic',  # Use 'multi:softmax' for multi-class classification
    'eval_metric': 'logloss',  # Use 'mlogloss' for multi-class
    'eta': 0.1,  # Learning rate
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'seed': 42
}

# Specify evaluation sets
evals = [(dtrain, 'train'), (dval, 'eval')]

# Train the model
num_rounds = 100
xgb_model = xgb.train(params, dtrain, num_rounds, evals, early_stopping_rounds=10, verbose_eval=True)

# Make predictions
y_pred_prob = xgb_model.predict(dval)
y_pred = np.round(y_pred_prob)  # Convert probabilities to binary (0 or 1)

# Evaluate accuracy
accuracy = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {accuracy:.4f}")

[0]	train-logloss:0.61745	eval-logloss:0.67761
[1]	train-logloss:0.58271	eval-logloss:0.67568
[2]	train-logloss:0.55366	eval-logloss:0.67151
[3]	train-logloss:0.52672	eval-logloss:0.66759
[4]	train-logloss:0.50416	eval-logloss:0.66614
[5]	train-logloss:0.48499	eval-logloss:0.66565
[6]	train-logloss:0.46111	eval-logloss:0.67934
[7]	train-logloss:0.44284	eval-logloss:0.69133
[8]	train-logloss:0.42676	eval-logloss:0.70398
[9]	train-logloss:0.41290	eval-logloss:0.71476
[10]	train-logloss:0.39951	eval-logloss:0.72964
[11]	train-logloss:0.38822	eval-logloss:0.73568
[12]	train-logloss:0.37862	eval-logloss:0.74767
[13]	train-logloss:0.37146	eval-logloss:0.75604
[14]	train-logloss:0.36385	eval-logloss:0.76120
[15]	train-logloss:0.35763	eval-logloss:0.76239
Validation Accuracy: 0.6300


/opt/anaconda3/envs/Phishing/lib/python3.12/site-packages/xgboost/core.py:723: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


In [173]:
# Convert X_val_encoded into DMatrix
dval = xgb.DMatrix(X_val_encoded)

# Make predictions (returns probabilities)
y_pred_prob = xgb_model.predict(dval)

# Convert probabilities to binary labels (0 or 1)
y_pred = np.round(y_pred_prob)

In [176]:
report1 =classification_report(y_val,y_pred)
report =classification_report(y_val,y_pred,output_dict=True)
# Print detailed classification report
print(report1)

              precision    recall  f1-score   support

           0       0.66      0.82      0.73      1132
           1       0.55      0.33      0.42       733

    accuracy                           0.63      1865
   macro avg       0.60      0.58      0.57      1865
weighted avg       0.61      0.63      0.61      1865



In [177]:
with mlflow.start_run(run_name="XG-boost"):
    mlflow.log_params({'n_estimators':100})
    mlflow.log_metrics({
        'accuracy ':report['accuracy'],
        'recall_class_1 ': report['1']['recall'],
        'f1_score_macro':report['macro avg']['f1-score'],
        'f1_score_macro':report['weighted avg']['f1-score']

    })
    mlflow.sklearn.log_model(nb_model,"naive_bayes")

2025/02/04 21:00:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XG-boost at: http://127.0.0.1:5000/#/experiments/618310494445782914/runs/075624dfdd424e5c9f4de3ff97e9fa2a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/618310494445782914
